# Creating A Pipeline Using MLRUN

In [1]:
%env MLRUN_DBPATH=/User/mlrun
import kfp
from kfp import dsl
from mlrun import run_start, get_run_db
from mlrun.platforms import mount_v3io
from mlrun.builder import build_image

env: MLRUN_PACKAGE_PATH=git+https://github.com/mlrun/mlrun.git@development


<b> Test/Debug the code locally, verify its working <b>

In [2]:
!python -m mlrun run -p p1=5 -s file=secrets.txt --out-path /User/mlrun/ training.py

Run: train (uid=980ac78c8ec14654a1587405d8f13ab8)
Params: p1=5, p2=a-string
accesskey = 1234
file
b"i'm a local input file"


Run: train (uid=980ac78c8ec14654a1587405d8f13ab8)
Params: p1=5, p2=a-string
accesskey = 1234
file
b"i'm a local input file"


[mlrun] 2019-09-04 21:24:03,064 run finished, status=completed 


## Build & Run a KubeFlow Pipeline 

This example is using iguazio shared FS (v3io), the `/User` dir is the "Home" for the user and the Jupyter notebook<br>
the code is mounted into the pipeline containers (no need to rebuild containers when the code changes and the runtime have access to the user local files)

MLRUN has a DB specified in the `db_path` argument (this example is using files to store runs and artifacts)<br>
the result artifacts are versioned and stored under the specified location, each workflow have a unique artifacts directory (`/<path>/{{workflow.uid}}/`)

Artifact and DB paths can use file paths or URLs for supported datastores (prefixed with s3://, v3io://, ..), <br>
Notes: file store artifacts cannot be viewed by KFP (use object URLs), URL based stores may requieres secrets passing 

In [12]:
this_path = '/User/mlrun'
db_path = this_path
#artifacts_path = this_path + '/data/{{workflow.uid}}/'
artifacts_path = 'v3io:///bigdata/mlrun/{{workflow.uid}}/'
image = 'mlrun/mypipe:latest'

# Build a container image

In [ ]:
# build an image with MLRUN (dev branch vs master), can add other packages to it 
# since registry secret was not specified it will push to the local cluster registry 
build_image(image, base_image='python:3.6')

## Example: 2 step workflow (training, validation), with hyper parameters
* 1st step: Execute training job with parameters <b>p1</b> and <b>p2</b>, log results and various artifacts including model (see [training.py](training.py))
* 2nd step: take the <b>modelfile</b> from the 1st stepand conduct validation (see [validation.py](validation.py))

### Building a Pipeline with Hyperparams and Parallel Execution
We may want to run the same training job with multiple parameter options, we can lavarage MLRUN paralelism<br>
, instead or running each run in a seperate container with extra start and stop times we can use a pool of serverless functions<br>
or containers which will run the workload in parallel.

We extend our pipeline to use hyper parameters, the training Job will accept a list per parameter and will run all the parameter<br>
combinations (GridSearch), involving the fixes parameters `params` and the expended parameters (from `hyperparams`)<br>
we can apply a `selector` which will return the best result based on the criteria e.g. `max.accuracy`.

Parameter combinations can also be provided using the `param_file` option which reads the parameter values per iteration<br>
from a CSV file (where the first row hold the parameter names and following rows hold param values).<br>
the use of `hyperparams` and `param_file` can be extended to many tasks including data and ETL tasks<br>
e.g. create a list of text or image file paths in a CSV file and run a step which process all those files in paralell. 

In [7]:
# run training using params p1 and p2, generate 2 registered outputs (model, dataset) to be listed in the pipeline UI
# user can specify the target path per output e.g. 'model.txt':'<some-path>', or leave blank to use the default out_path
def mlrun_train(p1, p2):
    return mlrun_op('training', 
                    image = image,
                    command = this_path + '/training.py', 
                    params = {'p2':p2},
                    hyperparams = {'p1': p1},
                    selector = 'max.accuracy',
                    in_path='/User/mlrun',
                    outputs = {'model.txt':'', 'dataset.csv':''},
                    out_path = artifacts_path)
                    
# use data (model) from the first step as an input
def mlrun_validate(modelfile):
    return mlrun_op('validation', 
                    image = image,
                    command = this_path + '/validation.py', 
                    inputs = {'model.txt':modelfile},
                    in_path='/User/mlrun',
                    out_path = artifacts_path)

<b> Create a Kubeflow Pipelines DSL (execution graph/DAG)</b>

In [8]:
@dsl.pipeline(
    name='My MLRUN pipeline',
    description='Shows how to use mlrun.'
)
def mlrun_pipeline(
   p1 = [5, 6, 2] , p2 = '"text"'
):
    # create a train step, apply v3io mount to it (will add the /User mount to the container)
    train = mlrun_train(p1, p2).apply(mount_v3io())
    
    # feed 1st step results into the secound step
    # Note: the '.' in model.txt must be substituted with '-'
    validate = mlrun_validate(train.outputs['model-txt']).apply(mount_v3io())

<b> Create a KFP client, Experiment and run the pipeline with custom parameter </b>

In [7]:
client = kfp.Client(namespace='default-tenant')
arguments = {'p1': [5, 7, 3]}
run_result = client.create_run_from_pipeline_func(mlrun_pipeline, arguments, run_name='mlrun demo 1', experiment_name='mlrun demo')

# for debug you can see the pipeline as a yaml file, use
# kfp.compiler.Compiler().compile(mlrun_pipeline, 'mlrunpipe.yaml')

<b> See the run status and results in the run database </b>

In [11]:
# connect to the run db 
db = get_run_db(db_path).connect()

In [17]:
# query the DB with filter on workflow ID (only show this workflow) 
db.list_runs('', labels=f'workflow={run_result.run_id}').show()

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...d6e7b3,0,Sep 01 23:26:43,completed,validation,workflow=c43690c7-0854-4bd2-b96b-24bbf3926246kind=localowner=roothost=my-mlrun-pipeline-zrh6n-1045808081,model.txt,,,validation.html
...1bd295,0,Sep 01 23:26:34,completed,training,workflow=c43690c7-0854-4bd2-b96b-24bbf3926246kind=localowner=roothost=my-mlrun-pipeline-zrh6n-1669727288framework=sklearn,infile.txt,p1=4p2=text,accuracy=8loss=12,model.txtresults.htmldataset.csvchart.html
